In [35]:
import cv2
import mediapipe as mp
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

# Inicializar MediaPipe
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Captura de video
cam = cv2.VideoCapture(0)

# Variables para mantener el estado
operacion_actual = ""
resultado = ""
ultimo_numero = ""
tiempo_ultima_deteccion = time.time()
intervalo_deteccion = 3 
tiempo_reinicio = 5  # Tiempo de reinicio cuando hay espera
mostrar_resultado = False  # Bandera para bloquear la entrada después del "="


chuleta_imagenes = [
    cv2.imread('SUMA.jpg'),
    cv2.imread('RESTA.jpg'),
    cv2.imread('MULTIPLICACION.jpg'),
    cv2.imread('DIVISION.jpg'),
    cv2.imread('IGUAL.jpg')
]

chuleta_imagenes = [cv2.resize(img, (100, 100)) for img in chuleta_imagenes]

while True:
    success, img = cam.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks and not mostrar_resultado:
        for handLms in results.multi_hand_landmarks:
            # Coordenadas de los puntos relevantes de la mano
            landmarks = handLms.landmark

            # Determinar si los dedos están levantados
            dedos_levantados = {
                "pulgar": landmarks[4].x > landmarks[3].x,
                "indice": landmarks[8].y < landmarks[6].y,
                "medio": landmarks[12].y < landmarks[10].y,
                "anular": landmarks[16].y < landmarks[14].y,
                "meñique": landmarks[20].y < landmarks[18].y,
            }

            # Contar los dedos levantados que no sean el meñique ni el pulgar
            dedos_detectados = sum([
                dedos_levantados["indice"],
                dedos_levantados["medio"],
                dedos_levantados["anular"]
            ]) + dedos_levantados["pulgar"] + dedos_levantados["meñique"]

            # Tiempo actual para el temporizador
            tiempo_actual = time.time()

            # RESTAR
            if (
                dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "-"
                resultado += ultimo_numero + "-"
                ultimo_numero = ""

            #SUMAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "+"
                resultado += ultimo_numero + "+"
                ultimo_numero = ""
            
            #MULTIPLICAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "*"
                resultado += ultimo_numero + "*"
                ultimo_numero = ""
            
            #DIVIDIR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "/"
                resultado += ultimo_numero + "/"
                ultimo_numero = ""

            #IGUAL
            elif (
                dedos_levantados["indice"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                resultado += ultimo_numero
                operacion_actual += "="

                try:
                    calculo = eval(resultado)
                    resultado += f"={calculo}"
                except:
                    resultado += "=Error"
                ultimo_numero = ""
                operacion_actual = ""
                mostrar_resultado = True  # Bloquear después de "="

            elif (
                dedos_detectados >= 0
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                if dedos_detectados == 0:
                    ultimo_numero += "0"
                elif dedos_detectados == 1:
                    ultimo_numero += "1"
                elif dedos_detectados == 2:
                    ultimo_numero += "2"
                elif dedos_detectados == 3:
                    ultimo_numero += "3"
                elif dedos_detectados == 4:
                    ultimo_numero += "4"
                elif dedos_detectados == 5:
                    ultimo_numero += "5"

            # Dibujar las conexiones de la mano
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

    # Si han pasado 5 segundos desde la última detección, reinicia la operación
    if time.time() - tiempo_ultima_deteccion > tiempo_reinicio:
        operacion_actual = ""
        resultado = ""
        ultimo_numero = ""
        mostrar_resultado = False  # Desbloquear entradas después del reinicio
    
    for i, imagen in enumerate(chuleta_imagenes):
        x_offset = 10 + i * 110  # Espacio entre imágenes
        # Coloca cada imagen en la parte superior de la ventana
        img[10:110, x_offset:x_offset + 100] = imagen

    # Mostrar el resultado en la pantalla
    cv2.putText(img, f"Operacion: {resultado}{ultimo_numero}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar imagen
    cv2.imshow("WebCam", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir con 'q'
        break

cam.release()
cv2.destroyAllWindows()


I0000 00:00:1735153818.965016 5980636 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
W0000 00:00:1735153819.138643 6112434 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735153819.193892 6112434 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [16]:
import cv2
import mediapipe as mp
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

# Inicializar MediaPipe
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Captura de video
cam = cv2.VideoCapture(0)

# Variables para mantener el estado
operacion_actual = ""
resultado = ""
ultimo_numero = ""
tiempo_ultima_deteccion = time.time()
intervalo_deteccion = 3 
tiempo_reinicio = 5  # Tiempo de reinicio cuando hay espera
mostrar_resultado = False  # Bandera para bloquear la entrada después del "="

# Función para contar los dedos levantados en una mano
def contar_dedos(landmarks, es_izquierda):
    # Evaluamos la posición de los dedos
    dedos_levantados = {
        "pulgar": landmarks[4].x < landmarks[3].x if es_izquierda else landmarks[4].x > landmarks[3].x,
        "indice": landmarks[8].y < landmarks[6].y,
        "medio": landmarks[12].y < landmarks[10].y,
        "anular": landmarks[16].y < landmarks[14].y,
        "meñique": landmarks[20].y < landmarks[18].y,
    }

    # Contar los dedos levantados
    dedos_detectados = sum([
        dedos_levantados["indice"],
        dedos_levantados["medio"],
        dedos_levantados["anular"]
    ]) + dedos_levantados["pulgar"] + dedos_levantados["meñique"]
    
    return dedos_detectados

while True:
    success, img = cam.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks and not mostrar_resultado:
        dedos_totales = 0  # Contador para la suma de dedos levantados en ambas manos
        
        for handLms in results.multi_hand_landmarks:
            # Coordenadas de los puntos relevantes de la mano
            landmarks = handLms.landmark
            
            # Determinar si la mano es la izquierda o derecha
            es_izquierda = landmarks[0].x < landmarks[9].x  # La coordenada x del punto 0 (muñeca) debe ser menor en la mano izquierda
            
            # Contar los dedos levantados para esta mano
            dedos_levantados = contar_dedos(landmarks, es_izquierda)
            dedos_totales += dedos_levantados  # Sumar los dedos de esta mano

            # Dibujar las conexiones de la mano
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

        # Procesamiento de las operaciones
        tiempo_actual = time.time()

        # Aquí irían las operaciones de suma, resta, etc. (igual que en tu código original)
        if dedos_totales > 0 and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion:
            tiempo_ultima_deteccion = tiempo_actual
            if dedos_totales == 1:
                ultimo_numero += "1"
            elif dedos_totales == 2:
                ultimo_numero += "2"
            elif dedos_totales == 3:
                ultimo_numero += "3"
            elif dedos_totales == 4:
                ultimo_numero += "4"
            elif dedos_totales == 5:
                ultimo_numero += "5"
            elif dedos_totales == 6:
                ultimo_numero += "6"
            elif dedos_totales == 7:
                ultimo_numero += "7"
            elif dedos_totales == 8:
                ultimo_numero += "8"
            elif dedos_totales == 9:
                ultimo_numero += "9"
            elif dedos_totales == 10:
                ultimo_numero += "10"

    # Si han pasado 5 segundos desde la última detección, reinicia la operación
    if time.time() - tiempo_ultima_deteccion > tiempo_reinicio:
        operacion_actual = ""
        resultado = ""
        ultimo_numero = ""
        mostrar_resultado = False  # Desbloquear entradas después del reinicio

    # Mostrar el resultado en la pantalla
    cv2.putText(img, f"Operacion: {resultado}{ultimo_numero}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar imagen
    cv2.imshow("WebCam", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir con 'q'
        break

cam.release()
cv2.destroyAllWindows()


I0000 00:00:1735152283.618132 5980636 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
W0000 00:00:1735152283.771447 6050306 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735152283.804153 6050306 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
